### Importing libraries

In [120]:
#general
import pandas as pd

#calculating geo distance
import geopandas as gpd
from shapely.geometry import Point

#importing data from URL
import requests

#allowing users to decide which functionality they prefer
import argparse

#allowing users to input interest points with flexibility 
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#for nice displaying
from tabulate import tabulate

### Importing function for calculating geo distance

In [2]:
# transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
def to_mercator(lat, long):  
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

# return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
def distance_meters(start, finish):
    return start.distance(finish)

### Extracting raw data - Conecting to BiciMad API

In [3]:
#It is possible to take frozen data from january 2022 using:
#bicimad_stations=pd.read_csv('../data/bicimad_stations.csv')
#bicimad_stations[["long_finish","lat_finish"]]=bicimad_stations['geometry_coordinates'].str.split(',', 1, expand=True)
#bicimad_stations["lat_finish"]=bicimad_stations["lat_finish"].str.replace("]","", regex=True)
#that was taken from a database
#but in order to have fresh data we include here the API connection

In [4]:
# getting token
login_url = 'https://openapi.emtmadrid.es/v1/mobilitylabs/user/login/'
login_headers = {'email': 'malva.gonzalez@ironhack.com','password': 'Bicimad2022'}
accessToken=requests.get(login_url,headers=login_headers).json()['data'][0]['accessToken']

In [5]:
#Getting stations information
stations_url = 'https://openapi.emtmadrid.es/v1/transport/bicimad/stations/'
stations_headers = {'accessToken':accessToken }
bicimad_stations_json=requests.get(stations_url,headers=stations_headers).json()['data']

In [7]:
bicimad_stations=pd.json_normalize(bicimad_stations_json)

Taking only the intersting columns needed for the project and filtering the data:

In [8]:
bicimad_stations=bicimad_stations[["address","geometry.coordinates","name","dock_bikes","free_bases"]]

In [9]:
bicimad_stations = bicimad_stations.rename(columns={'geometry.coordinates': 'geometry_coordinates'})

Filtering out rows without geometry_coordinates (distance can not be calculated, in this case there are not null values):

In [10]:
bicimad_stations=bicimad_stations[bicimad_stations.geometry_coordinates.notnull()]

Checking if stations name are unique (they are not but the address is different so this is not a problem):

In [11]:
bicimad_stations.groupby("name").filter(lambda x: len(x) > 1).sort_values("name")

,address,geometry_coordinates,name,dock_bikes,free_bases
129,Calle Fernando el Católico nº 19,"[-3.708439, 40.4338516]",Fernando el Católico,22,0
201,Calle Fernando el Católico nº 61,"[-3.7149166, 40.4343611]",Fernando el Católico,11,13
89,Calle Goya nº 18,"[-3.6837876, 40.4250361]",General Pardiñas,6,18
188,Calle General Pardiñas nº 56,"[-3.6782777, 40.4290555]",General Pardiñas,10,13
121,Calle Méndez Álvaro nº 11,"[-3.6863218, 40.4013216]",Méndez Álvaro,11,13
182,Calle Retama nº 5,"[-3.6758888, 40.3941388]",Méndez Álvaro,7,17
165,Paseo de la Esperanza nº 2,"[-3.7064516, 40.4035988]",Paseo de la Esperanza,9,5
172,Paseo de la Esperanza nº 21,"[-3.7043611, 40.401]",Paseo de la Esperanza,7,15


Calculating coordinates:

In [12]:
def left_split(x):
    return x[0]
def right_split(x):
    return x[-1]

In [13]:
long_finish=bicimad_stations.apply(lambda x: left_split(x['geometry_coordinates']), axis=1)

In [14]:
lat_finish=bicimad_stations.apply(lambda x: right_split(x['geometry_coordinates']), axis=1)

In [15]:
bicimad_stations["long_finish"]=long_finish

In [16]:
bicimad_stations["lat_finish"]=lat_finish

In [17]:
bicimad_stations

,address,geometry_coordinates,name,dock_bikes,free_bases,long_finish,lat_finish
0,Puerta del Sol nº 1,"[-3.7018341, 40.4172137]",Puerta del Sol A,11,17,-3.701834,40.417214
1,Puerta del Sol nº 1,"[-3.701602938060457, 40.41731271011562]",Puerta del Sol B,22,6,-3.701603,40.417313
2,Calle Miguel Moya nº 1,"[-3.7058415, 40.4205886]",Miguel Moya,9,14,-3.705842,40.420589
3,Plaza del Conde del Valle de Súchil nº 3,"[-3.7069171, 40.4302937]",Plaza Conde Suchil,8,7,-3.706917,40.430294
4,Calle Manuela Malasaña nº 5,"[-3.7025875, 40.4285524]",Malasaña,14,8,-3.702587,40.428552
...,...,...,...,...,...,...,...
259,Avenida Juan de Herrera frente a la calle Paul...,"[-3.72997, 40.43896]",INEF,3,19,-3.729970,40.438960
260,Avenida de la Complutense (Metro Ciudad Univer...,"[-3.72699, 40.44375]",Ciudad Universitaria 1,0,24,-3.726990,40.443750
261,Avenida de la Complutense (Metro Ciudad Univer...,"[-3.72693, 40.44342]",Ciudad Universitaria 2,5,18,-3.726930,40.443420
262,Calle José Antonio Novais frente al nº 12,"[-3.7272945, 40.4483322]",Facultad Biología,7,16,-3.727295,40.448332


Renaming field names:

In [18]:
bicimad_stations = bicimad_stations.rename(columns={'address': 'Station location', 'name': 'BiciMAD station'})

Deleting not needed columns:

In [19]:
bicimad_stations = bicimad_stations.drop(columns='geometry_coordinates')

Calculating coordenades:

In [20]:
bicimad_stations[["long_finish", "lat_finish"]] = bicimad_stations[["long_finish", 
                                                                   "lat_finish"]].apply(pd.to_numeric)

In [21]:
%%time
coordinates_bicimad=bicimad_stations.apply(lambda x: to_mercator(x['lat_finish'],
                                                              x['long_finish']), axis=1)

CPU times: user 6.42 s, sys: 533 ms, total: 6.95 s
Wall time: 6.99 s


In [22]:
bicimad_stations["coordinates_bicimad"]=coordinates_bicimad

In [23]:
bicimad_stations.drop(['long_finish','lat_finish'], axis=1, inplace=True)

In [24]:
bicimad_stations

,Station location,BiciMAD station,dock_bikes,free_bases,coordinates_bicimad
0,Puerta del Sol nº 1,Puerta del Sol A,11,17,POINT (4499223.648366921 -412373.2851572837)
1,Puerta del Sol nº 1,Puerta del Sol B,22,6,POINT (4499234.670122574 -412347.498528782)
2,Calle Miguel Moya nº 1,Miguel Moya,9,14,POINT (4499599.340516399 -412820.3206106317)
3,Plaza del Conde del Valle de Súchil nº 3,Plaza Conde Suchil,8,7,POINT (4500679.707306497 -412940.3068148262)
4,Calle Manuela Malasaña nº 5,Malasaña,14,8,POINT (4500485.866677178 -412457.3286497012)
...,...,...,...,...,...
259,Avenida Juan de Herrera frente a la calle Paul...,INEF,3,19,POINT (4501644.435409558 -415511.9577777153)
260,Avenida de la Complutense (Metro Ciudad Univer...,Ciudad Universitaria 1,0,24,POINT (4502177.655770458 -415179.5220676512)
261,Avenida de la Complutense (Metro Ciudad Univer...,Ciudad Universitaria 2,5,18,POINT (4502140.920338497 -415172.8287427756)
262,Calle José Antonio Novais frente al nº 12,Facultad Biología,7,16,POINT (4502687.743941171 -415213.4906984274)


### Extracting raw data -  Interesting Points

Defining public url for accesing to the datasets:

In [26]:
url_interest_points = 'https://datos.madrid.es/egob/catalogo/300356-0-monumentos-ciudad-madrid.json'

Taking url information with requests:

In [27]:
response_interest_points = requests.get(url_interest_points)

In [28]:
response_interest_points=response_interest_points.json()['@graph']

Converting json to pandas dataframe and taking only needed fields:

In [29]:
interest_points = pd.json_normalize(response_interest_points)

In [30]:
interest_points=interest_points[["title","address.street-address","location.latitude","location.longitude"]]

Renaming and adding columns needed:

In [31]:
interest_points = interest_points.rename(columns=
{'title': 'Place of interest', 'address.street-address': 'Place address',
 'location.latitude': 'lat_start','location.longitude': 'long_start'})

In [32]:
interest_points["Type of place"] = "Monumentos de la ciudad de Madrid" 

In [33]:
interest_points

,Place of interest,Place address,lat_start,long_start,Type of place
0,A las víctimas del Holocausto,JARDIN TRES CULTURAS,40.464322,-3.603558,Monumentos de la ciudad de Madrid
1,A los Abuelos,C Alicún,40.479812,-3.661719,Monumentos de la ciudad de Madrid
2,A Neruda-El Ser alado,Gta José Luis Ozores 116,40.381855,-3.652419,Monumentos de la ciudad de Madrid
3,Abogados Laboristas,C Atocha 55,40.412911,-3.700017,Monumentos de la ciudad de Madrid
4,Abstracta I,,40.421746,-3.662293,Monumentos de la ciudad de Madrid
...,...,...,...,...,...
1843,XXV aniversario Colegio Mayor Argentino,C Martín Fierro,40.436127,-3.731997,Monumentos de la ciudad de Madrid
1844,XXV aniversario Colegio Mayor Argentino,C Martín Fierro,40.436100,-3.732009,Monumentos de la ciudad de Madrid
1845,Yawar Fiesta,,NaN,NaN,Monumentos de la ciudad de Madrid
1846,'Sin título',,NaN,NaN,Monumentos de la ciudad de Madrid


Cleaning the dataframe (excluding rows without latitude or longitude information):

In [34]:
interest_points=interest_points[interest_points.lat_start.notnull()]

In [35]:
interest_points=interest_points[interest_points.long_start.notnull()]

Cheking if points of interest are unique (names are not unique so minim level of detail is name+address):

In [36]:
interest_points.groupby("Place of interest").filter(lambda x: len(x) > 1).sort_values("Place of interest")

,Place of interest,Place address,lat_start,long_start,Type of place
12,"Agricultor, agricultura y progreso",Avda Complutense,40.441972,-3.729384,Monumentos de la ciudad de Madrid
13,"Agricultor, agricultura y progreso",C Principal de Provincias,40.412532,-3.739214,Monumentos de la ciudad de Madrid
14,"Agricultor, agricultura y progreso",PASEO CASTELLANA 101,40.453458,-3.691112,Monumentos de la ciudad de Madrid
37,Alfonso X El Sabio,PASEO RECOLETOS 20,40.423004,-3.690836,Monumentos de la ciudad de Madrid
38,Alfonso X El Sabio,JARDINES SABATINI,40.421540,-3.713312,Monumentos de la ciudad de Madrid
...,...,...,...,...,...
1823,Vicente Aleixandre,Gta Presidente García Moreno,40.445849,-3.714497,Monumentos de la ciudad de Madrid
1843,XXV aniversario Colegio Mayor Argentino,C Martín Fierro,40.436127,-3.731997,Monumentos de la ciudad de Madrid
1844,XXV aniversario Colegio Mayor Argentino,C Martín Fierro,40.436100,-3.732009,Monumentos de la ciudad de Madrid
50,Álvaro Iglesias Sánchez,CALLE CARRANZA 7,40.429000,-3.703131,Monumentos de la ciudad de Madrid


Cheking if there are duplicates by name and address (6 duplicates, there will be cleaned later):

In [37]:
interest_duplicates=interest_points.groupby(["Place of interest","Place address"]).filter(lambda x: len(x) > 1).sort_values("Place of interest")[["Place of interest","Place address"]]

In [38]:
interest_duplicates

,Place of interest,Place address
70,Antonio Bienvenida,C Alcalá 231
71,Antonio Bienvenida,C Alcalá 231
173,Casa de la Panadería,Pza Mayor 27
174,Casa de la Panadería,Pza Mayor 27
340,Enrique Tierno Galván,
343,Enrique Tierno Galván,
1356,Miguel de Cervantes,C Cervantes 2
1357,Miguel de Cervantes,C Cervantes 2
1686,Rey astur-leonés,C Méndez Núñez 1
1687,Rey astur-leonés,C Méndez Núñez 1


In [39]:
(interest_duplicates["Place of interest"]+interest_duplicates["Place address"]).unique()

array(['Antonio BienvenidaC Alcalá 231',
       'Casa de la PanaderíaPza Mayor 27', 'Enrique Tierno Galván ',
       'Miguel de CervantesC Cervantes 2',
       'Rey astur-leonésC Méndez Núñez 1',
       'XXV aniversario Colegio Mayor ArgentinoC Martín Fierro '],
      dtype=object)

Calculating coordinades:

In [40]:
interest_points[["long_start", "lat_start"]] = interest_points[["long_start", 
                                                                   "lat_start"]].apply(pd.to_numeric)

In [41]:
%%time
coordinates_interest_points=interest_points.apply(lambda x: to_mercator(x['lat_start'],
                                                              x['long_start']), axis=1)

CPU times: user 42.7 s, sys: 3.06 s, total: 45.7 s
Wall time: 46 s


In [42]:
interest_points["coordinates_interest_points"]=coordinates_interest_points

In [43]:
interest_points.drop(['long_start','lat_start'], axis=1, inplace=True)

In [44]:
interest_points

,Place of interest,Place address,Type of place,coordinates_interest_points
0,A las víctimas del Holocausto,JARDIN TRES CULTURAS,Monumentos de la ciudad de Madrid,POINT (4504467.690585745 -401410.9402809891)
1,A los Abuelos,C Alicún,Monumentos de la ciudad de Madrid,POINT (4506192.045251724 -407898.4486445754)
2,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumentos de la ciudad de Madrid,POINT (4495287.585088818 -406861.0266344676)
3,Abogados Laboristas,C Atocha 55,Monumentos de la ciudad de Madrid,POINT (4498744.636240896 -412170.5648424985)
4,Abstracta I,,Monumentos de la ciudad de Madrid,POINT (4499728.13308047 -407962.4875817323)
...,...,...,...,...
1840,Wamba,Pza Oriente,Monumentos de la ciudad de Madrid,POINT (4499284.447377896 -413608.0839695063)
1841,Wifredo. El Velloso,Pza Oriente,Monumentos de la ciudad de Madrid,POINT (4499398.509382567 -413488.3182058548)
1842,X aniversario Colegio Mayor Argentino Nuestra ...,C Martín Fierro,Monumentos de la ciudad de Madrid,POINT (4501320.015285811 -415743.259706655)
1843,XXV aniversario Colegio Mayor Argentino,C Martín Fierro,Monumentos de la ciudad de Madrid,POINT (4501329.073807199 -415738.0970327701)


### Joining Data

Joining both tables with all posibilities for calculating the distance:

In [45]:
interest_points_bicimad = bicimad_stations.assign(key=0).merge(interest_points.assign(key=0), how='left', on = 'key')

In [46]:
interest_points_bicimad.drop('key', axis=1, inplace=True)

In [47]:
interest_points_bicimad

,Station location,BiciMAD station,dock_bikes,free_bases,coordinates_bicimad,Place of interest,Place address,Type of place,coordinates_interest_points
0,Puerta del Sol nº 1,Puerta del Sol A,11,17,POINT (4499223.648366921 -412373.2851572837),A las víctimas del Holocausto,JARDIN TRES CULTURAS,Monumentos de la ciudad de Madrid,POINT (4504467.690585745 -401410.9402809891)
1,Puerta del Sol nº 1,Puerta del Sol A,11,17,POINT (4499223.648366921 -412373.2851572837),A los Abuelos,C Alicún,Monumentos de la ciudad de Madrid,POINT (4506192.045251724 -407898.4486445754)
2,Puerta del Sol nº 1,Puerta del Sol A,11,17,POINT (4499223.648366921 -412373.2851572837),A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumentos de la ciudad de Madrid,POINT (4495287.585088818 -406861.0266344676)
3,Puerta del Sol nº 1,Puerta del Sol A,11,17,POINT (4499223.648366921 -412373.2851572837),Abogados Laboristas,C Atocha 55,Monumentos de la ciudad de Madrid,POINT (4498744.636240896 -412170.5648424985)
4,Puerta del Sol nº 1,Puerta del Sol A,11,17,POINT (4499223.648366921 -412373.2851572837),Abstracta I,,Monumentos de la ciudad de Madrid,POINT (4499728.13308047 -407962.4875817323)
...,...,...,...,...,...,...,...,...,...
473083,Avenida Complutense nº 23,Facultad Derecho,13,10,POINT (4502994.74083288 -415445.024322674),Wamba,Pza Oriente,Monumentos de la ciudad de Madrid,POINT (4499284.447377896 -413608.0839695063)
473084,Avenida Complutense nº 23,Facultad Derecho,13,10,POINT (4502994.74083288 -415445.024322674),Wifredo. El Velloso,Pza Oriente,Monumentos de la ciudad de Madrid,POINT (4499398.509382567 -413488.3182058548)
473085,Avenida Complutense nº 23,Facultad Derecho,13,10,POINT (4502994.74083288 -415445.024322674),X aniversario Colegio Mayor Argentino Nuestra ...,C Martín Fierro,Monumentos de la ciudad de Madrid,POINT (4501320.015285811 -415743.259706655)
473086,Avenida Complutense nº 23,Facultad Derecho,13,10,POINT (4502994.74083288 -415445.024322674),XXV aniversario Colegio Mayor Argentino,C Martín Fierro,Monumentos de la ciudad de Madrid,POINT (4501329.073807199 -415738.0970327701)


### Calculating distance

In [48]:
%%time
distance=interest_points_bicimad.apply(lambda x: distance_meters(x['coordinates_interest_points'],
                                                              x['coordinates_bicimad']), axis=1)

CPU times: user 6.43 s, sys: 67.4 ms, total: 6.5 s
Wall time: 6.54 s


In [49]:
interest_points_bicimad["distance"]=distance

Deleting not needed columns:

In [50]:
interest_points_bicimad.drop(['coordinates_bicimad','coordinates_interest_points'], axis=1, inplace=True)

Sorting values:

In [51]:
interest_points_bicimad=interest_points_bicimad.sort_values(["Place of interest","Station location"])

In [52]:
interest_points_bicimad

,Station location,BiciMAD station,dock_bikes,free_bases,Place of interest,Place address,Type of place,distance
437250,Avenida Bruselas nº 56,Avenida Bruselas,9,13,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumentos de la ciudad de Madrid,6431.780052
471298,Avenida Complutense nº 23,Facultad Derecho,13,10,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumentos de la ciudad de Madrid,11536.258751
464130,Avenida Juan de Herrera frente a la calle Paul...,INEF,3,19,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumentos de la ciudad de Madrid,10735.369376
421122,Avenida Marqués de Corbera nº 12,Marqués de Corbera 12,12,12,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumentos de la ciudad de Madrid,4589.330269
422914,Avenida Marqués de Corbera nº 52,Marqués de Corbera 52,9,13,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumentos de la ciudad de Madrid,4939.620431
...,...,...,...,...,...,...,...,...
64163,Plaza del Cordón,Plaza del Cordón,16,8,Órgano,C Castelló 77,Monumentos de la ciudad de Madrid,3759.110762
1443,Puerta del Sol nº 1,Puerta del Sol A,11,17,Órgano,C Castelló 77,Monumentos de la ciudad de Madrid,2776.677372
3235,Puerta del Sol nº 1,Puerta del Sol B,22,6,Órgano,C Castelló 77,Monumentos de la ciudad de Madrid,2749.128671
92835,Ronda de Atocha nº 2,Ronda de Atocha,14,10,Órgano,C Castelló 77,Monumentos de la ciudad de Madrid,2962.047768


### Filtering by the minimun distant:

In [65]:
nearest_BiciMAD_station=interest_points_bicimad[interest_points_bicimad['distance'] ==
                        interest_points_bicimad.groupby(['Place of interest','Place address'])['distance'].transform('min')]

In [66]:
nearest_BiciMAD_station

,Station location,BiciMAD station,dock_bikes,free_bases,Place of interest,Place address,Type of place,distance
410370,Calle Arroyo del Olivar nº 6,Concordia,21,1,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumentos de la ciudad de Madrid,2012.572727
415744,Calle Carlota O'Neill nº 32,Carlota O’Neill,5,18,A las víctimas del Holocausto,JARDIN TRES CULTURAS,Monumentos de la ciudad de Madrid,5559.887276
449793,Paseo de la Castellana nº 298,Manuel Caldeiro,7,16,A los Abuelos,C Alicún,Monumentos de la ciudad de Madrid,2633.107767
164883,Calle Goya nº 1,Plaza de Colón,0,20,AICO,Pza Colón,Monumentos de la ciudad de Madrid,110.126957
73475,Calle Atocha nº 54,Antón Martín,16,0,Abogados Laboristas,C Atocha 55,Monumentos de la ciudad de Madrid,127.684089
...,...,...,...,...,...,...,...,...
62769,Plaza del Cordón,Plaza del Cordón,16,8,Álvaro de Bazán,Pza Villa,Monumentos de la ciudad de Madrid,102.328354
453441,Calle Santa Hortensia nº 31,Corazón de María,7,16,Ángeles de la Paz,Avda Bruselas,Monumentos de la ciudad de Madrid,326.569317
415829,Calle Carlota O'Neill nº 32,Carlota O’Neill,5,18,Árbol de la Vida,JARDIN TRES CULTURAS,Monumentos de la ciudad de Madrid,5559.887276
65596,Plaza de Ramales,Plaza Ramales,6,17,Íñigo Arista,Pza Oriente,Monumentos de la ciudad de Madrid,108.278072


Example for testing:

In [67]:
nearest_BiciMAD_station[(nearest_BiciMAD_station["Place of interest"]=="A las víctimas del Holocausto") | (nearest_BiciMAD_station["Place of interest"]=="A los Abuelos")]

,Station location,BiciMAD station,dock_bikes,free_bases,Place of interest,Place address,Type of place,distance
415744,Calle Carlota O'Neill nº 32,Carlota O’Neill,5,18,A las víctimas del Holocausto,JARDIN TRES CULTURAS,Monumentos de la ciudad de Madrid,5559.887276
449793,Paseo de la Castellana nº 298,Manuel Caldeiro,7,16,A los Abuelos,C Alicún,Monumentos de la ciudad de Madrid,2633.107767


Sorting the data:

In [68]:
nearest_BiciMAD_station=nearest_BiciMAD_station.sort_values(["Place of interest","Station location"])
nearest_BiciMAD_station=nearest_BiciMAD_station.reset_index()

In [69]:
nearest_BiciMAD_station

,index,Station location,BiciMAD station,dock_bikes,free_bases,Place of interest,Place address,Type of place,distance
0,410370,Calle Arroyo del Olivar nº 6,Concordia,21,1,A Neruda-El Ser alado,Gta José Luis Ozores 116,Monumentos de la ciudad de Madrid,2012.572727
1,415744,Calle Carlota O'Neill nº 32,Carlota O’Neill,5,18,A las víctimas del Holocausto,JARDIN TRES CULTURAS,Monumentos de la ciudad de Madrid,5559.887276
2,449793,Paseo de la Castellana nº 298,Manuel Caldeiro,7,16,A los Abuelos,C Alicún,Monumentos de la ciudad de Madrid,2633.107767
3,164883,Calle Goya nº 1,Plaza de Colón,0,20,AICO,Pza Colón,Monumentos de la ciudad de Madrid,110.126957
4,73475,Calle Atocha nº 54,Antón Martín,16,0,Abogados Laboristas,C Atocha 55,Monumentos de la ciudad de Madrid,127.684089
...,...,...,...,...,...,...,...,...,...
1781,62769,Plaza del Cordón,Plaza del Cordón,16,8,Álvaro de Bazán,Pza Villa,Monumentos de la ciudad de Madrid,102.328354
1782,453441,Calle Santa Hortensia nº 31,Corazón de María,7,16,Ángeles de la Paz,Avda Bruselas,Monumentos de la ciudad de Madrid,326.569317
1783,415829,Calle Carlota O'Neill nº 32,Carlota O’Neill,5,18,Árbol de la Vida,JARDIN TRES CULTURAS,Monumentos de la ciudad de Madrid,5559.887276
1784,65596,Plaza de Ramales,Plaza Ramales,6,17,Íñigo Arista,Pza Oriente,Monumentos de la ciudad de Madrid,108.278072


In [70]:
column_names = ["Place of interest", "Type of place", "Place address",
                "BiciMAD station","Station location","distance","dock_bikes","free_bases"]
nearest_BiciMAD_station=nearest_BiciMAD_station.reindex(columns=column_names)

In [71]:
nearest_BiciMAD_station = nearest_BiciMAD_station.rename(columns={'dock_bikes': 'bikes availability','distance': 'distance(m)','free_bases': 'bases availability'})

In [72]:
nearest_BiciMAD_station

,Place of interest,Type of place,Place address,BiciMAD station,Station location,distance(m),bikes availability,bases availability
0,A Neruda-El Ser alado,Monumentos de la ciudad de Madrid,Gta José Luis Ozores 116,Concordia,Calle Arroyo del Olivar nº 6,2012.572727,21,1
1,A las víctimas del Holocausto,Monumentos de la ciudad de Madrid,JARDIN TRES CULTURAS,Carlota O’Neill,Calle Carlota O'Neill nº 32,5559.887276,5,18
2,A los Abuelos,Monumentos de la ciudad de Madrid,C Alicún,Manuel Caldeiro,Paseo de la Castellana nº 298,2633.107767,7,16
3,AICO,Monumentos de la ciudad de Madrid,Pza Colón,Plaza de Colón,Calle Goya nº 1,110.126957,0,20
4,Abogados Laboristas,Monumentos de la ciudad de Madrid,C Atocha 55,Antón Martín,Calle Atocha nº 54,127.684089,16,0
...,...,...,...,...,...,...,...,...
1781,Álvaro de Bazán,Monumentos de la ciudad de Madrid,Pza Villa,Plaza del Cordón,Plaza del Cordón,102.328354,16,8
1782,Ángeles de la Paz,Monumentos de la ciudad de Madrid,Avda Bruselas,Corazón de María,Calle Santa Hortensia nº 31,326.569317,7,16
1783,Árbol de la Vida,Monumentos de la ciudad de Madrid,JARDIN TRES CULTURAS,Carlota O’Neill,Calle Carlota O'Neill nº 32,5559.887276,5,18
1784,Íñigo Arista,Monumentos de la ciudad de Madrid,Pza Oriente,Plaza Ramales,Plaza de Ramales,108.278072,6,17


In [73]:
nearest_BiciMAD_station['distance(m)']=nearest_BiciMAD_station['distance(m)'].round(2)

In [74]:
nearest_BiciMAD_station

,Place of interest,Type of place,Place address,BiciMAD station,Station location,distance(m),bikes availability,bases availability
0,A Neruda-El Ser alado,Monumentos de la ciudad de Madrid,Gta José Luis Ozores 116,Concordia,Calle Arroyo del Olivar nº 6,2012.57,21,1
1,A las víctimas del Holocausto,Monumentos de la ciudad de Madrid,JARDIN TRES CULTURAS,Carlota O’Neill,Calle Carlota O'Neill nº 32,5559.89,5,18
2,A los Abuelos,Monumentos de la ciudad de Madrid,C Alicún,Manuel Caldeiro,Paseo de la Castellana nº 298,2633.11,7,16
3,AICO,Monumentos de la ciudad de Madrid,Pza Colón,Plaza de Colón,Calle Goya nº 1,110.13,0,20
4,Abogados Laboristas,Monumentos de la ciudad de Madrid,C Atocha 55,Antón Martín,Calle Atocha nº 54,127.68,16,0
...,...,...,...,...,...,...,...,...
1781,Álvaro de Bazán,Monumentos de la ciudad de Madrid,Pza Villa,Plaza del Cordón,Plaza del Cordón,102.33,16,8
1782,Ángeles de la Paz,Monumentos de la ciudad de Madrid,Avda Bruselas,Corazón de María,Calle Santa Hortensia nº 31,326.57,7,16
1783,Árbol de la Vida,Monumentos de la ciudad de Madrid,JARDIN TRES CULTURAS,Carlota O’Neill,Calle Carlota O'Neill nº 32,5559.89,5,18
1784,Íñigo Arista,Monumentos de la ciudad de Madrid,Pza Oriente,Plaza Ramales,Plaza de Ramales,108.28,6,17


### Saving a csv with the results:

In [75]:
nearest_BiciMAD_station.to_csv('../data/nearest_BiciMAD_station.csv')

### Function for selecting one interesting place

In [76]:
def nearest_station_specific_place(place):
    nearest_BiciMAD_station_filtered=nearest_BiciMAD_station[nearest_BiciMAD_station["Place of interest"]==place]
    nearest_BiciMAD_station_filtered.to_csv('../data/nearest_BiciMAD_station_filtered.csv')
    return nearest_BiciMAD_station_filtered

Example:

In [77]:
nearest_station_specific_place(place='A los Abuelos')

,Place of interest,Type of place,Place address,BiciMAD station,Station location,distance(m),bikes availability,bases availability
2,A los Abuelos,Monumentos de la ciudad de Madrid,C Alicún,Manuel Caldeiro,Paseo de la Castellana nº 298,2633.11,7,16


### Using fuzzywuzzy

In [109]:
#Using an example for testing functionality
input_var='monumento a los abuelos'

In [110]:
def compare_strings(input_var,x):
    return fuzz.ratio(input_var,x)

In [111]:
similarity=nearest_BiciMAD_station.apply(lambda x: compare_strings(x['Place of interest'],input_var), axis=1)

In [112]:
similarity

0       27
1       35
2       61
3        0
4       43
        ..
1781    16
1782    20
1783    26
1784    23
1785    14
Length: 1786, dtype: int64

In [113]:
nearest_BiciMAD_station["similarity"]=similarity

In [114]:
nearest_BiciMAD_station

,Place of interest,Type of place,Place address,BiciMAD station,Station location,distance(m),bikes availability,bases availability,similarity
0,A Neruda-El Ser alado,Monumentos de la ciudad de Madrid,Gta José Luis Ozores 116,Concordia,Calle Arroyo del Olivar nº 6,2012.57,21,1,27
1,A las víctimas del Holocausto,Monumentos de la ciudad de Madrid,JARDIN TRES CULTURAS,Carlota O’Neill,Calle Carlota O'Neill nº 32,5559.89,5,18,35
2,A los Abuelos,Monumentos de la ciudad de Madrid,C Alicún,Manuel Caldeiro,Paseo de la Castellana nº 298,2633.11,7,16,61
3,AICO,Monumentos de la ciudad de Madrid,Pza Colón,Plaza de Colón,Calle Goya nº 1,110.13,0,20,0
4,Abogados Laboristas,Monumentos de la ciudad de Madrid,C Atocha 55,Antón Martín,Calle Atocha nº 54,127.68,16,0,43
...,...,...,...,...,...,...,...,...,...
1781,Álvaro de Bazán,Monumentos de la ciudad de Madrid,Pza Villa,Plaza del Cordón,Plaza del Cordón,102.33,16,8,16
1782,Ángeles de la Paz,Monumentos de la ciudad de Madrid,Avda Bruselas,Corazón de María,Calle Santa Hortensia nº 31,326.57,7,16,20
1783,Árbol de la Vida,Monumentos de la ciudad de Madrid,JARDIN TRES CULTURAS,Carlota O’Neill,Calle Carlota O'Neill nº 32,5559.89,5,18,26
1784,Íñigo Arista,Monumentos de la ciudad de Madrid,Pza Oriente,Plaza Ramales,Plaza de Ramales,108.28,6,17,23


In [115]:
nearest_BiciMAD_station[(nearest_BiciMAD_station["similarity"]>60)]

,Place of interest,Type of place,Place address,BiciMAD station,Station location,distance(m),bikes availability,bases availability,similarity
2,A los Abuelos,Monumentos de la ciudad de Madrid,C Alicún,Manuel Caldeiro,Paseo de la Castellana nº 298,2633.11,7,16,61
727,Fuente de los Caballos,Monumentos de la ciudad de Madrid,C Arnedo 22,Marqués de Vadillo,Glorieta Marqués de Vadillo nº 9,3109.16,16,5,67
728,Fuente de los Caños,Monumentos de la ciudad de Madrid,GLORIETA CAÑOS,Puerta del Ángel,Plaza Puerta del Ángel nº 4,3597.36,19,1,62
739,Fuente de los Patos,Monumentos de la ciudad de Madrid,Pº Prado,Cibeles,Plaza Cibeles,64.04,11,10,62
1373,Monumento a la Paz,Monumentos de la ciudad de Madrid,,Carlota O’Neill,Calle Carlota O'Neill nº 32,4300.75,5,18,63
1376,Monumento a los caídos,Monumentos de la ciudad de Madrid,GLORIETA EJERCITO 1,Caramuel,Calle Caramuel nº 51,3020.03,17,6,80
1377,Monumento a los caídos de Fuencarral,Monumentos de la ciudad de Madrid,PLAZA MADRE MARIA ANA MOGAS 2,Pedro Rico,Calle Pedro Rico nº 4,1480.62,2,20,61
